# Notebook 111: Momentum と Nesterov 加速

## Momentum SGD and Nesterov Accelerated Gradient

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第2章として、基本的なSGDの問題を解決する **Momentum** と **Nesterov加速** を学びます。

### 学習目標

1. **Momentum** の直感と数式を理解する
2. **Nesterov Accelerated Gradient (NAG)** の仕組みを理解する
3. SGDとの比較を通じて改善効果を確認する
4. 各手法の実装と可視化

### 前提知識

- Notebook 110 の内容（勾配降下法の基礎）

---

## 目次

1. [SGDの問題点](#1-sgdの問題点)
2. [Momentum SGD](#2-momentum-sgd)
3. [Nesterov Accelerated Gradient](#3-nesterov-accelerated-gradient)
4. [実装と比較](#4-実装と比較)
5. [様々な損失曲面での挙動](#5-様々な損失曲面での挙動)
6. [ハイパーパラメータの影響](#6-ハイパーパラメータの影響)
7. [演習問題](#7-演習問題)
8. [まとめと次のステップ](#8-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. SGDの問題点

### 1.1 振動問題

急峻な谷（細長い楕円形の等高線）を持つ損失関数では、SGDは谷の壁に沿って行ったり来たりしながらゆっくりと進みます。

In [ ]:
# 病的な損失関数（急峻な谷）

def pathological_loss(x, y, condition_number=50):
    """条件数の大きい楕円形損失関数"""
    return x**2 + condition_number * y**2

def pathological_grad(x, y, condition_number=50):
    """勾配"""
    return np.array([2*x, 2*condition_number*y])


def sgd_optimize(grad_fn, start, lr=0.01, n_steps=100, **kwargs):
    """基本的なSGD"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    
    for _ in range(n_steps):
        grad = grad_fn(pos[0], pos[1], **kwargs)
        pos = pos - lr * grad
        path.append(pos.copy())
    
    return np.array(path)


# SGDの振動を可視化
start = (3, 1)
path_sgd = sgd_optimize(pathological_grad, start, lr=0.01, n_steps=100, condition_number=50)

# 等高線
x = np.linspace(-4, 4, 200)
y = np.linspace(-2, 2, 200)
X, Y = np.meshgrid(x, y)
Z = pathological_loss(X, Y, condition_number=50)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'ro-', markersize=3, linewidth=1, alpha=0.7)
axes[0].scatter([0], [0], color='green', s=100, marker='*', zorder=5, label='最適点')
axes[0].scatter([start[0]], [start[1]], color='blue', s=100, zorder=5, label='開始点')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('SGDの振動問題（条件数 = 50）')
axes[0].legend()
axes[0].set_xlim(-4, 4)
axes[0].set_ylim(-2, 2)

# パラメータの推移
axes[1].plot(path_sgd[:, 0], label='x', linewidth=2)
axes[1].plot(path_sgd[:, 1], label='y', linewidth=2)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('パラメータ値')
axes[1].set_title('パラメータの収束')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【問題点】")
print("- y方向: 勾配が大きいため激しく振動")
print("- x方向: 勾配が小さいためゆっくり進む")
print("- 全体として収束が遅い")

### 1.2 条件数と収束速度

**条件数** (Condition Number) は、ヘッセ行列の最大固有値と最小固有値の比です：

$$
\kappa = \frac{\lambda_{\max}}{\lambda_{\min}}
$$

条件数が大きいほど、SGDの収束は遅くなります。

In [ ]:
# 条件数と収束速度の関係
condition_numbers = [1, 10, 50, 100]

fig, ax = plt.subplots(figsize=(10, 6))

for cn in condition_numbers:
    path = sgd_optimize(pathological_grad, (3, 1), lr=0.01, n_steps=200, condition_number=cn)
    losses = [pathological_loss(p[0], p[1], cn) for p in path]
    ax.semilogy(losses, linewidth=2, label=f'条件数 κ = {cn}')

ax.set_xlabel('ステップ')
ax.set_ylabel('損失（対数スケール）')
ax.set_title('条件数と収束速度の関係')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("- 条件数が大きいほど収束が遅い")
print("- κ = 100 では、κ = 1 の約100倍のステップが必要")

---

## 2. Momentum SGD

### 2.1 アイデア

物理学のアナロジー：ボールが斜面を転がる様子を想像してください。

- ボールは **速度（モーメンタム）** を持つ
- 一度方向が決まると、すぐには方向転換しない
- これにより振動が抑制され、一貫した方向への移動が促進される

### 2.2 数式

$$
\begin{aligned}
v_t &= \mu v_{t-1} + \eta \nabla L(\theta_t) \\
\theta_{t+1} &= \theta_t - v_t
\end{aligned}
$$

- $v$: 速度（velocity）
- $\mu$: モーメンタム係数（通常 0.9）
- 過去の勾配の指数移動平均を使用

In [ ]:
def momentum_optimize(grad_fn, start, lr=0.01, momentum=0.9, n_steps=100, **kwargs):
    """
    Momentum SGD
    
    v = μ * v + η * ∇L
    θ = θ - v
    """
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    velocity = np.zeros_like(pos)
    
    for _ in range(n_steps):
        grad = grad_fn(pos[0], pos[1], **kwargs)
        velocity = momentum * velocity + lr * grad
        pos = pos - velocity
        path.append(pos.copy())
    
    return np.array(path)


# Momentumの直感的な理解
print("【Momentumの効果】")
print("")
print("速度の更新: v = μ * v + η * ∇L")
print("")
print("過去の勾配の重み付け（μ = 0.9 の場合）:")
weights = [0.9**i for i in range(10)]
for i, w in enumerate(weights):
    bar = "█" * int(w * 20)
    print(f"  t-{i}: {w:.4f} {bar}")
print(f"")
print(f"  合計: {sum(weights):.2f}（μ/(1-μ) = {0.9/0.1:.1f}）")

In [ ]:
# SGD vs Momentum の比較
start = (3, 1)

path_sgd = sgd_optimize(pathological_grad, start, lr=0.01, n_steps=100, condition_number=50)
path_momentum = momentum_optimize(pathological_grad, start, lr=0.01, momentum=0.9, n_steps=100, condition_number=50)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'r.-', markersize=3, linewidth=1, alpha=0.7, label='SGD')
axes[0].plot(path_momentum[:, 0], path_momentum[:, 1], 'b.-', markersize=3, linewidth=1, alpha=0.7, label='Momentum')
axes[0].scatter([0], [0], color='green', s=100, marker='*', zorder=5, label='最適点')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('SGD vs Momentum')
axes[0].legend()
axes[0].set_xlim(-4, 4)
axes[0].set_ylim(-2, 2)

# 損失の推移
losses_sgd = [pathological_loss(p[0], p[1], 50) for p in path_sgd]
losses_momentum = [pathological_loss(p[0], p[1], 50) for p in path_momentum]

axes[1].semilogy(losses_sgd, 'r-', linewidth=2, label='SGD')
axes[1].semilogy(losses_momentum, 'b-', linewidth=2, label='Momentum')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【結果】")
print(f"SGD      最終損失: {losses_sgd[-1]:.6f}")
print(f"Momentum 最終損失: {losses_momentum[-1]:.6f}")
print(f"")
print("Momentumは振動を抑制し、より効率的に収束")

### 2.3 Momentumが振動を抑制する理由

1. **一貫した方向の勾配は加速される**
   - 同じ方向の勾配が続くと、速度が積み重なる

2. **振動方向の勾配は相殺される**
   - 符号が交互に変わる勾配は、速度の更新で打ち消し合う

In [ ]:
# 振動の抑制メカニズムを可視化

# シミュレーション: 一定方向 vs 交互方向
n_steps = 20
mu = 0.9

# ケース1: 一貫した勾配（x方向）
grads_consistent = np.ones(n_steps)
velocity_consistent = np.zeros(n_steps + 1)
for t in range(n_steps):
    velocity_consistent[t + 1] = mu * velocity_consistent[t] + grads_consistent[t]

# ケース2: 振動する勾配（y方向）
grads_oscillating = np.array([1 if i % 2 == 0 else -1 for i in range(n_steps)])
velocity_oscillating = np.zeros(n_steps + 1)
for t in range(n_steps):
    velocity_oscillating[t + 1] = mu * velocity_oscillating[t] + grads_oscillating[t]

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

steps = range(n_steps + 1)

axes[0].bar(range(n_steps), grads_consistent, alpha=0.5, label='勾配')
axes[0].plot(steps, velocity_consistent, 'b-', linewidth=2, marker='o', label='速度')
axes[0].set_xlabel('ステップ')
axes[0].set_ylabel('値')
axes[0].set_title('一貫した勾配 → 速度が加速')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].bar(range(n_steps), grads_oscillating, alpha=0.5, label='勾配')
axes[1].plot(steps, velocity_oscillating, 'r-', linewidth=2, marker='o', label='速度')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('値')
axes[1].set_title('振動する勾配 → 速度は相殺される')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print(f"一貫した勾配の最終速度: {velocity_consistent[-1]:.2f}")
print(f"振動する勾配の最終速度: {velocity_oscillating[-1]:.2f}")

---

## 3. Nesterov Accelerated Gradient

### 3.1 アイデア

Nesterov Accelerated Gradient (NAG) は、Momentumを改良したものです。

**Momentumの問題点**:
- 現在位置で勾配を計算してから移動する
- 移動先で実際に必要な勾配とはずれている可能性がある

**Nesterovの解決策**:
- まず「移動予定先」を計算（lookahead）
- その位置で勾配を計算
- より正確な勾配情報が得られる

### 3.2 数式

$$
\begin{aligned}
v_t &= \mu v_{t-1} + \eta \nabla L(\theta_t - \mu v_{t-1}) \\
\theta_{t+1} &= \theta_t - v_t
\end{aligned}
$$

勾配を計算する位置が $\theta_t$ ではなく $\theta_t - \mu v_{t-1}$（先読み位置）になっています。

In [ ]:
def nesterov_optimize(grad_fn, start, lr=0.01, momentum=0.9, n_steps=100, **kwargs):
    """
    Nesterov Accelerated Gradient (NAG)
    
    1. 先読み位置を計算: θ_lookahead = θ - μ * v
    2. 先読み位置で勾配を計算: g = ∇L(θ_lookahead)
    3. 速度を更新: v = μ * v + η * g
    4. パラメータを更新: θ = θ - v
    """
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    velocity = np.zeros_like(pos)
    
    for _ in range(n_steps):
        # 先読み位置
        lookahead = pos - momentum * velocity
        # 先読み位置で勾配を計算
        grad = grad_fn(lookahead[0], lookahead[1], **kwargs)
        # 速度更新
        velocity = momentum * velocity + lr * grad
        # パラメータ更新
        pos = pos - velocity
        path.append(pos.copy())
    
    return np.array(path)


# Nesterovの先読みを可視化
fig, ax = plt.subplots(figsize=(10, 6))

# 1次元の例
x = np.linspace(-3, 3, 100)
y = x**2  # 二次関数

ax.plot(x, y, 'b-', linewidth=2)

# 現在位置
current_x = 2.0
current_y = current_x**2
velocity = 0.5  # 仮の速度
mu = 0.9

# 先読み位置
lookahead_x = current_x - mu * velocity
lookahead_y = lookahead_x**2

# プロット
ax.scatter([current_x], [current_y], color='blue', s=150, zorder=5, label='現在位置 θ')
ax.scatter([lookahead_x], [lookahead_y], color='red', s=150, zorder=5, label='先読み位置 θ - μv')

# 矢印
ax.annotate('', xy=(lookahead_x, lookahead_y), xytext=(current_x, current_y),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.text((current_x + lookahead_x)/2 + 0.1, (current_y + lookahead_y)/2 + 0.3, 'μv (モーメンタム)', fontsize=11)

# 勾配を表示
grad_current = 2 * current_x
grad_lookahead = 2 * lookahead_x

ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_title('Nesterovの先読み（Lookahead）')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"現在位置での勾配: {grad_current:.2f}")
print(f"先読み位置での勾配: {grad_lookahead:.2f}")
print("")
print("Nesterovは先読み位置の勾配を使用することで、")
print("より正確な更新方向を得ることができます。")

---

## 4. 実装と比較

### 4.1 3手法の比較

In [ ]:
# SGD vs Momentum vs Nesterov
start = (3, 1)
n_steps = 100

path_sgd = sgd_optimize(pathological_grad, start, lr=0.01, n_steps=n_steps, condition_number=50)
path_momentum = momentum_optimize(pathological_grad, start, lr=0.01, momentum=0.9, n_steps=n_steps, condition_number=50)
path_nesterov = nesterov_optimize(pathological_grad, start, lr=0.01, momentum=0.9, n_steps=n_steps, condition_number=50)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'r.-', markersize=2, linewidth=1, alpha=0.7, label='SGD')
axes[0].plot(path_momentum[:, 0], path_momentum[:, 1], 'b.-', markersize=2, linewidth=1, alpha=0.7, label='Momentum')
axes[0].plot(path_nesterov[:, 0], path_nesterov[:, 1], 'g.-', markersize=2, linewidth=1, alpha=0.7, label='Nesterov')
axes[0].scatter([0], [0], color='black', s=100, marker='*', zorder=5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('最適化軌跡の比較')
axes[0].legend()
axes[0].set_xlim(-1, 4)
axes[0].set_ylim(-1.5, 1.5)

# 損失の推移
losses_sgd = [pathological_loss(p[0], p[1], 50) for p in path_sgd]
losses_momentum = [pathological_loss(p[0], p[1], 50) for p in path_momentum]
losses_nesterov = [pathological_loss(p[0], p[1], 50) for p in path_nesterov]

axes[1].semilogy(losses_sgd, 'r-', linewidth=2, label='SGD')
axes[1].semilogy(losses_momentum, 'b-', linewidth=2, label='Momentum')
axes[1].semilogy(losses_nesterov, 'g-', linewidth=2, label='Nesterov')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【最終損失】")
print(f"SGD:      {losses_sgd[-1]:.6f}")
print(f"Momentum: {losses_momentum[-1]:.6f}")
print(f"Nesterov: {losses_nesterov[-1]:.6f}")

### 4.2 PyTorchスタイルの実装

In [ ]:
class SGDOptimizer:
    """PyTorchスタイルのSGDオプティマイザ"""
    
    def __init__(self, params, lr=0.01, momentum=0.0, nesterov=False):
        """
        Args:
            params: パラメータのリスト
            lr: 学習率
            momentum: モーメンタム係数
            nesterov: Nesterovモーメンタムを使用するか
        """
        self.params = params
        self.lr = lr
        self.momentum = momentum
        self.nesterov = nesterov
        self.velocities = [np.zeros_like(p) for p in params]
    
    def step(self, grads):
        """パラメータを更新"""
        for i, (param, grad, v) in enumerate(zip(self.params, grads, self.velocities)):
            if self.momentum > 0:
                v = self.momentum * v + grad
                self.velocities[i] = v
                
                if self.nesterov:
                    update = self.momentum * v + grad
                else:
                    update = v
            else:
                update = grad
            
            param -= self.lr * update


# 使用例
params = [np.array([3.0, 1.0])]

# 各オプティマイザをテスト
optimizers = {
    'SGD': SGDOptimizer(params=[np.array([3.0, 1.0])], lr=0.01),
    'Momentum': SGDOptimizer(params=[np.array([3.0, 1.0])], lr=0.01, momentum=0.9),
    'Nesterov': SGDOptimizer(params=[np.array([3.0, 1.0])], lr=0.01, momentum=0.9, nesterov=True),
}

print("SGDOptimizer クラスを定義しました")
print("")
print("使用方法:")
print("  optimizer = SGDOptimizer(params, lr=0.01, momentum=0.9, nesterov=True)")
print("  optimizer.step(grads)  # 勾配でパラメータを更新")

---

## 5. 様々な損失曲面での挙動

### 5.1 Rosenbrock関数

In [ ]:
def rosenbrock(x, y):
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])


# 各手法での最適化
start = (-1.5, 1.5)
n_steps = 1000

path_sgd = sgd_optimize(rosenbrock_grad, start, lr=0.001, n_steps=n_steps)
path_momentum = momentum_optimize(rosenbrock_grad, start, lr=0.001, momentum=0.9, n_steps=n_steps)
path_nesterov = nesterov_optimize(rosenbrock_grad, start, lr=0.001, momentum=0.9, n_steps=n_steps)

# 等高線
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X_r, Y_r = np.meshgrid(x, y)
Z_r = rosenbrock(X_r, Y_r)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 軌跡
axes[0].contour(X_r, Y_r, np.log(Z_r + 1), levels=30, cmap='viridis', alpha=0.7)
axes[0].plot(path_sgd[:, 0], path_sgd[:, 1], 'r-', linewidth=1, alpha=0.7, label='SGD')
axes[0].plot(path_momentum[:, 0], path_momentum[:, 1], 'b-', linewidth=1, alpha=0.7, label='Momentum')
axes[0].plot(path_nesterov[:, 0], path_nesterov[:, 1], 'g-', linewidth=1, alpha=0.7, label='Nesterov')
axes[0].scatter([1], [1], color='black', s=100, marker='*', zorder=5, label='最適点')
axes[0].scatter([start[0]], [start[1]], color='orange', s=100, zorder=5, label='開始点')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Rosenbrock関数での最適化')
axes[0].legend()

# 損失の推移
losses_sgd = [rosenbrock(p[0], p[1]) for p in path_sgd]
losses_momentum = [rosenbrock(p[0], p[1]) for p in path_momentum]
losses_nesterov = [rosenbrock(p[0], p[1]) for p in path_nesterov]

axes[1].semilogy(losses_sgd, 'r-', linewidth=2, label='SGD')
axes[1].semilogy(losses_momentum, 'b-', linewidth=2, label='Momentum')
axes[1].semilogy(losses_nesterov, 'g-', linewidth=2, label='Nesterov')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('収束速度の比較')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.2 鞍点（Saddle Point）での挙動

In [ ]:
def saddle(x, y):
    return x**2 - y**2

def saddle_grad(x, y):
    return np.array([2*x, -2*y])


# 鞍点付近からの最適化
start = (0.1, 0.1)
n_steps = 50

path_sgd = sgd_optimize(saddle_grad, start, lr=0.1, n_steps=n_steps)
path_momentum = momentum_optimize(saddle_grad, start, lr=0.1, momentum=0.9, n_steps=n_steps)
path_nesterov = nesterov_optimize(saddle_grad, start, lr=0.1, momentum=0.9, n_steps=n_steps)

# 等高線
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X_s, Y_s = np.meshgrid(x, y)
Z_s = saddle(X_s, Y_s)

fig, ax = plt.subplots(figsize=(10, 8))

contour = ax.contour(X_s, Y_s, Z_s, levels=20, cmap='coolwarm')
ax.plot(path_sgd[:, 0], path_sgd[:, 1], 'ro-', markersize=4, linewidth=2, label='SGD')
ax.plot(path_momentum[:, 0], path_momentum[:, 1], 'bs-', markersize=4, linewidth=2, label='Momentum')
ax.plot(path_nesterov[:, 0], path_nesterov[:, 1], 'g^-', markersize=4, linewidth=2, label='Nesterov')
ax.scatter([0], [0], color='black', s=200, marker='X', zorder=5, label='鞍点')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('鞍点での挙動')
ax.legend()
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

plt.colorbar(contour)
plt.tight_layout()
plt.show()

print("【観察】")
print("- SGD: 鞍点に吸い込まれやすい")
print("- Momentum/Nesterov: 慣性により鞍点を通過しやすい")

---

## 6. ハイパーパラメータの影響

### 6.1 モーメンタム係数の影響

In [ ]:
# モーメンタム係数の比較
momentum_values = [0.0, 0.5, 0.9, 0.99]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for mu in momentum_values:
    if mu == 0:
        path = sgd_optimize(pathological_grad, (3, 1), lr=0.01, n_steps=100, condition_number=50)
    else:
        path = momentum_optimize(pathological_grad, (3, 1), lr=0.01, momentum=mu, n_steps=100, condition_number=50)
    
    losses = [pathological_loss(p[0], p[1], 50) for p in path]
    axes[0].semilogy(losses, linewidth=2, label=f'μ = {mu}')

axes[0].set_xlabel('ステップ')
axes[0].set_ylabel('損失（対数スケール）')
axes[0].set_title('モーメンタム係数と収束')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 効果的な学習率の計算
mu_range = np.linspace(0, 0.99, 100)
effective_lr = 1 / (1 - mu_range)  # 定常状態での実効学習率

axes[1].plot(mu_range, effective_lr, 'b-', linewidth=2)
axes[1].set_xlabel('モーメンタム係数 μ')
axes[1].set_ylabel('実効学習率倍率 1/(1-μ)')
axes[1].set_title('モーメンタムによる実効学習率の増加')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, 20)

# 典型的な値をマーク
for mu in [0.9, 0.99]:
    eff = 1 / (1 - mu)
    axes[1].scatter([mu], [eff], s=100, zorder=5)
    axes[1].annotate(f'μ={mu}: {eff:.0f}x', (mu, eff), textcoords="offset points", xytext=(10, 10))

plt.tight_layout()
plt.show()

print("【モーメンタム係数の指針】")
print("μ = 0.9:  一般的な選択（実効10倍）")
print("μ = 0.99: より強いモーメンタム（実効100倍）")
print("")
print("注意: μが大きいと発散のリスクも高まる")

### 6.2 学習率とモーメンタムの組み合わせ

In [ ]:
# 学習率とモーメンタムの組み合わせ
lrs = [0.001, 0.01, 0.05]
mus = [0.0, 0.9]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for i, mu in enumerate(mus):
    for j, lr in enumerate(lrs):
        ax = axes[i, j]
        
        if mu == 0:
            path = sgd_optimize(pathological_grad, (3, 1), lr=lr, n_steps=100, condition_number=50)
        else:
            path = momentum_optimize(pathological_grad, (3, 1), lr=lr, momentum=mu, n_steps=100, condition_number=50)
        
        ax.contour(X, Y, Z, levels=30, cmap='viridis', alpha=0.5)
        ax.plot(path[:, 0], path[:, 1], 'r.-', markersize=2, linewidth=1)
        ax.scatter([0], [0], color='black', s=50, marker='*')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        
        final_loss = pathological_loss(path[-1, 0], path[-1, 1], 50)
        title = f'lr={lr}, μ={mu}\n最終損失: {final_loss:.4f}'
        ax.set_title(title)
        ax.set_xlim(-1, 4)
        ax.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

---

## 7. 演習問題

### 演習 7.1: Beale関数での最適化

Beale関数で SGD, Momentum, Nesterov を比較してください。

$$
f(x, y) = (1.5 - x + xy)^2 + (2.25 - x + xy^2)^2 + (2.625 - x + xy^3)^2
$$

最適点: $(3, 0.5)$

In [ ]:
# 演習 7.1: 解答欄

def beale(x, y):
    # TODO: 実装
    pass

def beale_grad(x, y):
    # TODO: 実装
    pass

# TODO: 3手法を比較
pass

### 演習 7.2: モーメンタム係数のチューニング

病的な損失関数（条件数=100）に対して、最も効率的に収束するモーメンタム係数を見つけてください。

In [ ]:
# 演習 7.2: 解答欄

# TODO: μ = 0.5, 0.7, 0.9, 0.95, 0.99 を比較

pass

### 演習 7.3: Nesterovの優位性

Nesterovが Momentum より優れているケースを見つけてください。
ヒント: 最適点をオーバーシュートしそうな状況で違いが顕著になります。

In [ ]:
# 演習 7.3: 解答欄

# TODO: Nesterovの先読みが効果的なケースを探す

pass

---

## 8. まとめと次のステップ

### このノートブックで学んだこと

1. **SGDの問題点**: 急峻な谷での振動、収束の遅さ

2. **Momentum SGD**:
   - 速度（過去の勾配の指数移動平均）を導入
   - 一貫した方向への移動を加速
   - 振動を抑制

3. **Nesterov Accelerated Gradient**:
   - 先読み位置で勾配を計算
   - オーバーシュートを軽減
   - Momentumよりやや速い収束

4. **ハイパーパラメータ**:
   - モーメンタム係数 $\mu$: 通常 0.9
   - 学習率は実効的に $1/(1-\mu)$ 倍される

### 次のノートブック（112: 適応学習率手法）への橋渡し

Momentum/Nesterovは以下の問題を解決しません：

- **パラメータごとの学習率**: すべてのパラメータに同じ学習率を適用
- **学習率のチューニング**: 適切な学習率の選択が困難

次のノートブックでは、これらを解決する **適応学習率手法**（Adagrad, RMSprop, Adam）を学びます。

---

## 参考文献

1. Polyak, B. T. (1964). Some methods of speeding up the convergence of iteration methods. *USSR Computational Mathematics and Mathematical Physics*.
2. Nesterov, Y. (1983). A method for solving the convex programming problem with convergence rate O(1/k²). *Soviet Mathematics Doklady*.
3. Sutskever, I., et al. (2013). On the importance of initialization and momentum in deep learning. *ICML*.